# remove folder

In [3]:
import os
from glob import glob
import shutil

In [ ]:
path = '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/南一/answer'

In [8]:
path_list = glob('/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/*/*/*')
path_list

['/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/翰林/answer/pdf',
 '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/翰林/question/pdf',
 '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/南一/answer/pdf',
 '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/南一/question/pdf',
 '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/康軒/answer/pdf',
 '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/康軒/question/pdf']

In [9]:
# for path in path_list:
#     if path.split('/')[-1] != 'pdf':
#         print(path)
#         shutil.rmtree(path)

# pdf to images


In [3]:
import os
from glob import glob
from pdf2image import convert_from_path

In [4]:
class PdfUtil:
    def __init__(self):
        pass
    
    def rename(self, old):
        # 全形轉半形（ => (
        #old = '/home/ryh/embedding-match/ocr/datasets/taiwan/13to15/press/108/數學/南一/answer/pdf/108上[南一]國中試卷-數學五A卷-(優等)-3上（答）.pdf'
        new = old.replace('（', '(').replace('）', ')')
        os.rename(old, new)
    
    def get_pdf_path_list(self, year='*', subject='*', press='*', qa='*'):
        d = './datasets/%s/%s/%s/%s/pdf/*.pdf'%(year, subject, press, qa)
        pdf_path_list = sorted(glob(d))
        return pdf_path_list
    
    def pdf_to_images(self, pdf_path):
        images = convert_from_path(pdf_path)
        
        if self.is_pdf_already_process(pdf_path, images): 
            print(' ----------- already process ----------- ')
            print(pdf_path)
            return 

        old_dirname = os.path.dirname(pdf_path)
        old_basename = os.path.basename(pdf_path)

        new_dirname = old_dirname.replace('pdf', 'paper')
        os.makedirs(new_dirname, exist_ok=True)
        for i, image in enumerate(images):     
            #print(i+1, len(images), end='\r')
            new_basename = old_basename.replace('.pdf', '') + '---%02d'%(i+1) + '.png' 
            img_path = os.path.join(new_dirname, new_basename)
            image.save(img_path, 'PNG')
            
    def is_pdf_already_process(self, pdf_path, images):
        i = len(images) - 1
        
        old_dirname = os.path.dirname(pdf_path)
        old_basename = os.path.basename(pdf_path)
        new_dirname = old_dirname.replace('pdf', 'paper')
        new_basename = old_basename.replace('.pdf', '') + '---%02d'%(i+1) + '.png' 
        img_path = os.path.join(new_dirname, new_basename)
        return os.path.isfile(img_path) 
        

In [5]:
pdfUtil = PdfUtil()

## rename

In [ ]:
# 108上[南一]國中試卷-數學（一）-B卷-(中上)-答.pdf

In [6]:
pdf_list = pdfUtil.get_pdf_path_list(year='*', subject='數學', press='*', qa='*')
len(pdf_list)

118

In [9]:
# pdf_list

In [7]:
# for old in pdf_list:
#     pdfUtil.rename(old)

## pdf_to_images

In [11]:
# for i, pdf in enumerate(pdf_list):
#     print(i+1, len(pdf_list), end='\r')
#     pdfUtil.pdf_to_images(pdf)
    

# upload files to GCS

In [19]:
from google.cloud import storage
import os
from glob import glob

In [23]:
class GcsUtil:
    def __init__(self, bucket_name='easy-ocr2'):
        self.storage_client = storage.Client()
        self.bucket_name = bucket_name
        self.bucket = self.storage_client.bucket(self.bucket_name)
        self.setup_done_file()
    
    def get_paper_path_list(self, year='*', subject='*', press='*', qa='*'):
        d = './datasets/%s/%s/%s/%s/paper/*.png'%(year, subject, press, qa)
        paper_path_list = sorted(glob(d))
        return paper_path_list

    def setup_done_file(self):
        blobs = self.storage_client.list_blobs(self.bucket_name, prefix='datasets/')
        blob_list = list(blobs)
        file_list = [os.path.basename(blob.name) for blob in blob_list]        
        self.done_file = set(file_list)
    
    def upload_blob(self, local_path):         
        remote_path = local_path[local_path.index('datasets'):] # 刪掉datasets以前的字
        blob = self.bucket.blob(remote_path)
        
        if os.path.basename(local_path) in self.done_file:
            print(' ----------- already upload ----------- ')
            print(local_path)
            pass
        else:
            blob.upload_from_filename(local_path)
            
gcsUtil = GcsUtil(bucket_name='easy-ocr2')


## rename
- 假如pdf就改名了，這裡就不用做

In [24]:
# paper_path_list = gcsUtil.get_paper_path_list()
# len(paper_path_list)

In [25]:
# for old in paper_path_list:
#     pdfUtil.rename(old)

## upload_blob

In [26]:
paper_path_list = gcsUtil.get_paper_path_list()
len(paper_path_list)

2998

In [28]:
# for i, paper_path in enumerate(paper_path_list[:]):
#     print(i+1, len(paper_path_list), end='\r')
#     gcsUtil.upload_blob(paper_path)
    

# input_file.csv 
- https://cloud.google.com/ai-platform-unified/docs/datasets/prepare-image

In [ ]:
把所有paper扣掉已經在input_file裡面的

In [29]:
from google.cloud import storage
import pandas as pd
from glob import glob
import os
import re

In [92]:
class LabelUtil:
    def __init__(self, bucket_name='easy-ocr'):
        self.storage_client = storage.Client()
        self.bucket_name = bucket_name
        
    def get_all_uri(self):
        blobs = self.storage_client.list_blobs(self.bucket_name, prefix='datasets/')
        blob_list = list(blobs)
        uri_list = ['gs://' + self.bucket_name + '/' + blob.name for blob in blob_list]        
        return set(uri_list)
        
    def get_done_uri(self):
        done_uri = []
        path_list = glob('./input_file/*.csv')
        for path in path_list:
            with open(path, 'r') as f:
                uris = f.read().splitlines() 
                done_uri += uris
        return set(done_uri)
    
    def get_todo_url(self):
        done_uri = self.get_done_uri()
        all_uri = self.get_all_uri()
        todo_url = list(all_uri - done_uri)
        todo_url = sorted(todo_url)
        return todo_url
    
    def get_filter_uri(self, uri_list, year='*', subject='*', press='*', qa='*'):
        filter_uri = []
        
        for uri in uri_list:
            split_index = uri.index('datasets/') + len('datasets/')
            uri_ = uri[split_index:] # 刪掉datasets以前的字
            year_, subject_, press_, qa_, _, _ = uri_.split('/')
            c1 = year != '*' and year != year_
            c2 = subject != '*' and subject != subject_
            c3 = press != '*' and press != press_
            c4 = qa != '*' and qa != qa_
            if c1 or c2 or c3 or c4:
                continue
            filter_uri.append(uri)
        filter_uri = sorted(filter_uri)
        return filter_uri
    
    def save_uri(self, uri_list, save_path):
        df = pd.DataFrame()
        df['GCS_FILE_PATH'] = uri_list
        df.to_csv(save_path, header=False, index=False)
        
labelUtil = LabelUtil(bucket_name='easy-ocr2')

In [93]:
# labelUtil.get_done_uri()

In [94]:
todo_url = labelUtil.get_todo_url()
len(todo_url)

0

In [103]:
for year in ['108']:
    for subject in ['數學']: # ['數學']
        for press in ['南一', '康軒', '翰林']: # ['南一', '康軒', '翰林']
            for qa in ['*']:
                filter_uri = labelUtil.get_filter_uri(todo_url, year, subject, press, qa)
                num = len(filter_uri)
                save_path = './input_file/%s_%s_%s.csv'%(year, press, num)

                if num == 0: 
                    print('num = 0')
                    break
                
                if os.path.isfile(save_path) == False:
                    labelUtil.save_uri(filter_uri, save_path)

num = 0
num = 0
num = 0


# split image to segments

In [10]:
import os
import json
import chardet
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [25]:
class SegUtil:
    def __init__(self):
        pass
    
    def get_paperInfo_list(self, output_file_path):
        with open(output_file_path) as f:
            row_list = f.readlines()
        paperInfo_list = []
        for row in row_list:
            d = json.loads(row)
            imageGcsUri = d['imageGcsUri']
            imageGcsUri = imageGcsUri.encode('l1').decode('utf-8')
            imageGcsUri = imageGcsUri[imageGcsUri.index('datasets'):] # 刪掉datasets以前的字 
            imageGcsUri = os.path.join('.', imageGcsUri)
            d['imageGcsUri'] = imageGcsUri
            paperInfo_list.append(d)
        return paperInfo_list
    
    def paper_to_segments(self, paperInfo):
        paper_path = paperInfo['imageGcsUri']
        img = mpimg.imread(paper_path)
        img_height, img_width = img.shape[0], img.shape[1]
        
        old_dirname = os.path.dirname(paper_path)
        new_dirname = old_dirname.replace('paper', 'seg')
        old_basename = os.path.basename(paper_path)
        
        os.makedirs(new_dirname, exist_ok=True) #?
        
        if 'boundingBoxAnnotations' not in paperInfo: 
            print(' ----------- no box_list ----------- ')
            print(paper_path)
            return
        
        box_list = paperInfo['boundingBoxAnnotations']
        box_list = sorted(box_list, key=(lambda x: x['yMax']), reverse=True) # 從底部往上
        for box in box_list: # 假如沒有box 啥都沒做
            displayName = box['displayName']
            if 'w' in displayName:
                ch = displayName.split('w')[0] # 第幾回
            else:
                name = ch + '_' + displayName.replace('q', '_') # 14_4_2
                new_basename = old_basename.split('---')[0] + '---' + name + '.png' # 108上[南一]國中試卷-(三)數學-A卷-(優等)-(答)---01.png
                path = os.path.join(new_dirname, new_basename)
                
                if os.path.isfile(path):
                    #print(' ----------- already exist ----------- ')
                    #print(path)
                    continue

                xMax, xMin, yMax, yMin = box['xMax'], box['xMin'], box['yMax'], box['yMin']
                xMax, xMin = int(xMax*img_width), int(xMin*img_width)
                yMax, yMin = int(yMax*img_height), int(yMin*img_height)
                seg = img[yMin:yMax, xMin:xMax, :]
                
                plt.imsave(path, seg)

segUtil = SegUtil()


In [29]:
output_file_path = './output_file/output_file_export-data-108_康軒_1322-2021-04-01T16_02_48.388920Z_image_bounding_box_108_康軒_1322_iod-5893153626460782592_data-00001-of-00001.jsonl'
paperInfo_list = segUtil.get_paperInfo_list(output_file_path)
len(paperInfo_list)


1322

In [32]:
paperInfo = paperInfo_list[2]
# paperInfo


In [33]:
segUtil.paper_to_segments(paperInfo)

 ----------- no box_list ----------- 
./datasets/108/數學/康軒/answer/paper/108下-康軒國中試卷數學(4)C卷答---11.png


In [34]:
def decode_txt(txt):
    txt = txt.encode('l1').decode('utf-8')
    return txt

txt = '108ä¸-åº·è»åä¸­è©¦å·æ¸å­¸(4)Aå·(åªç­)ç­-2ä¸---18.png'
decode_txt(txt)

'108下-康軒國中試卷數學(4)A卷(優等)答-2下---18.png'